In [1]:
import pandas as pd
import numpy as np
import glob
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', -1)

import time
from pandas_profiling import ProfileReport

In [2]:
##################Print Multiple Cmds#####################
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# IO & PARSING FUNCTIONS

In [3]:
def read_multilog(f):
    lst = []
    with open(f, 'r', encoding='utf8') as filehandle:
        for line in filehandle:
            lst.append(line)
    return lst

In [4]:
def clean_multilog(logList):
    lst = []
    for item in range(len(logList)):
        temp = logList[item].replace('\n','')
        lst.append(temp)
    return lst

In [5]:
def split_pipes(logList):
    splitList = []
    for ii in range(len(logList)):
        temp = logList[ii].split('|')
    return temp

# COMBINE FUNCTIONS

In [6]:
def combine_logfiles(log_file_list):
    combined_logs = []
    filename_list = []
    for log_file in range(len(log_file_list)):
        read_list = read_multilog(log_file_list[log_file])
        temp = clean_multilog(read_list)
#         print(len(temp))
        if len(temp) == 1:
            temp1 = split_pipes(temp)
            combined_logs.append(temp1)
            filename_list.append(log_file_list[log_file])
        else:
            for item in range(len(temp)):
                combined_logs.append(temp[item].split('|'))
                filename_list.append(log_file_list[log_file])
    return combined_logs, filename_list

In [7]:
def split_equalSign_combine(combined_list, filename_list):
    df = pd.DataFrame()
    for item in range(len(combined_list)):
        fields = []
        values = []
        for record in range(len(combined_list[item])):
            temp_record = combined_list[item][record].split('=',1)
            fields.append(temp_record[0])
            values.append(temp_record[1])

        temp_df = pd.DataFrame(data=values).T
        temp_df.columns = fields
        df = df.append(temp_df, sort=False)

    df = pd.concat([pd.DataFrame(filename_list, columns=['Filename']), df.reset_index(drop=True)], axis=1)
    return df

In [8]:
def get_split_pathname(x):
    for k in range(x.shape[0]):
        x.Filename[k] = x.Filename[k].split('\\')[1]        
    return x

# Process logs

### segment mimecast traffic logfiles - receipt / process / delivery / jrnl

In [9]:
# traffic_receipt_logfiles = glob.glob('C:/Users/R011846/Desktop/full_logfiles_test/receipt*.log')
# print('receipt count:', len(traffic_receipt_logfiles))
# traffic_process_logfiles = glob.glob('C:/Users/R011846/Desktop/full_logfiles_test/process*.log')
# print('process count:', len(traffic_process_logfiles))
# traffic_delivery_logfiles = glob.glob('C:/Users/R011846/Desktop/full_logfiles_test/delivery*.log')
# print('delivery count:', len(traffic_delivery_logfiles))
# traffic_jrnl_logfiles = glob.glob('C:/Users/R011846/Desktop/full_logfiles_test/jrnl*.log')
# print('jrnl count:', len(traffic_jrnl_logfiles))

In [54]:
traffic_receipt_logfiles = glob.glob('C:/Users/R011846/Documents/GitHub/mimecast_log_collector/logs/log_files/receipt*.log')
print('receipt count:', len(traffic_receipt_logfiles))

receipt count: 3173


### complete logfile traffic

In [55]:
# log_file_list = glob.glob('C:/Users/R011846/Desktop/full_logfiles_test/*.log')
# print('all logs count:', len(log_file_list))

### get column of logfile traffic type values - receipt / process / delivery / jrnl

In [56]:
# temp_list = []
# for blah in range(len(log_file_list)):
#     temp_list.append(log_file_list[blah].split('\\')[1].split('_')[0])
# email_traffic_types = pd.DataFrame(data=save_me, columns=['log_type'])

# Combine Logs

### Receipt

In [57]:
combined_receipt_list, receipt_filename_list = combine_logfiles(traffic_receipt_logfiles)
print('\nreceipt clean list count:', len(combined_receipt_list))


receipt clean list count: 15688


### Process

In [58]:
# combined_process_list, process_filename_list = combine_logfiles(traffic_process_logfiles)
# print('\nprocess clean list count:', len(combined_process_list))

### Delivery

In [59]:
# combined_delivery_list, delivery_filename_list = combine_logfiles(traffic_delivery_logfiles)
# print('\ndelivery clean list count:', len(combined_delivery_list))

# Create & write combined dataframe

## Receipt

In [66]:
start = time.time()
df_receipt2 = split_equalSign_combine(combined_receipt_list, receipt_filename_list)
end = time.time()

print('elapsed time in seconds: {:0.1f}'.format(end - start))
print('final dataframe shape:', df_receipt.shape)

elapsed time in seconds: 370.7
final dataframe shape: (31308, 22)


In [67]:
df_receipt2 = get_split_pathname(df_receipt2)
df_receipt2.head(1)

,Filename,datetime,aCode,acc,SpamLimit,IP,Dir,MsgId,Subject,headerFrom,Sender,Rcpt,SpamInfo,Act,TlsVer,Cphr,SpamScore,RejType,RejCode,RejInfo,Error,Virus
0,receipt_20200206094427111.log,2020-02-06T09:13:14-0500,Y7CekpLRPPyY25DN0c3eZw,CUSA31A369,5,198.71.244.77,Inbound,<wyF1.139573.398072.25626173.1580997787.2456126.Ajb@a2plmmsworker11.prod.phx3.gdg.mail>,5 Business Process Optimization Tips,newsletters@finance-insights.us.com,sp_139573.25626173.1.2f121654b4cb1b0704784229e366ca78@bounces.em.secureserver.net,emurch@usventure.com,[],Acc,TLSv1.2,TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,4,NaN,NaN,NaN,NaN,NaN


- Example durations: 700s, n=31,308 
#### Output receipt pickle

In [64]:
pklName = 'all_receipt.pkl'
# df_receipt = get_split_pathname(pd.read_pickle(pklName))
# df_receipt.to_pickle(pklName)
df_receipt = pd.read_pickle(pklName)
df_receipt.shape

(31308, 22)

In [65]:
df_receipt[df_receipt.aCode == '1cep7I0JObKgYoyRv17sPQ'].Filename.value_counts()

receipt_20200203164052967.log    62
Name: Filename, dtype: int64

## Process

In [17]:
# start = time.time()
# df_process = split_equalSign_combine(combined_process_list, process_filename_list)
# end = time.time()

# print('elapsed time in seconds: {:0.1f}'.format(end - start))
# print('final dataframe shape:', df_process.shape)

- Example durations: 194s, n=24,303
#### Output process pickle

In [18]:
pklName = 'all_process.pkl'
# df_process = get_split_pathname(pd.read_pickle(pklName))
# df_process.to_pickle(pklName)
df_process = pd.read_pickle(pklName)
df_process.shape

(24303, 11)

## Delivery

In [19]:
# start = time.time()
# df_delivery = split_equalSign_combine(combined_delivery_list, delivery_filename_list)
# end = time.time()

# print('elapsed time in seconds: {:0.1f}'.format(end - start))
# print('final dataframe shape:', df_delivery.shape)

- Example durations: 574.2, n=25,555
#### Output delivery pickle

In [20]:
pklName = 'all_delivery.pkl'
# df_delivery = get_split_pathname(pd.read_pickle(pklName))
# df_delivery.to_pickle(pklName)
df_delivery = pd.read_pickle(pklName)
df_delivery.shape

(25555, 25)

# Process email logs direction - only INBOUND

## Receipt

In [62]:
df_receipt.Dir.value_counts()

df_inbound_rec = df_receipt[df_receipt.Dir == 'Inbound'].reset_index(drop=True)
# df_outbound = df_receipt[df_receipt.Dir == 'Outbound'].reset_index(drop=True)
# df_external = df_receipt[df_receipt.Dir == 'External'].reset_index(drop=True)
# df_internal = df_receipt[df_receipt.Dir == 'Internal'].reset_index(drop=True)
# df_inbound_rec.columns = df_inbound_rec.columns + '_receipt'
df_inbound_rec.columns

Inbound    15688
Name: Dir, dtype: int64

Index(['Filename', 'datetime', 'aCode', 'acc', 'SpamLimit', 'IP', 'Dir', 'MsgId', 'Subject', 'headerFrom', 'Sender', 'Rcpt', 'SpamInfo', 'Act', 'TlsVer', 'Cphr', 'SpamScore', 'RejType', 'RejCode', 'RejInfo', 'Error', 'Virus'], dtype='object')

In [22]:
# profile = ProfileReport(df_inbound_rec, title='Pandas Profiling Report - Mimecast Receipt')
# # # # profile.to_notebook_iframe()
# profile.to_file(output_file='mimecast_inbound_traffic_receipt.html')

In [23]:
# df_inbound_rec.to_csv('mimecast_inbound_traffic_receipt.csv', index=False)

## Process - NO DIRECTION - will try to link acodes
- There are duplicates in these files that will be removed - WHY?

In [24]:
df_process = df_process[~df_process.duplicated()]
df_process.shape
# df_process.columns = df_process.columns + '_process'
df_process.columns

(20057, 11)

Index(['Filename', 'datetime', 'aCode', 'acc', 'AttSize', 'Act', 'AttCnt', 'AttNames', 'MsgSize', 'MsgId', 'Hld'], dtype='object')

In [25]:
# profile = ProfileReport(df_process, title='Pandas Profiling Report - Mimecast Process')
# # # # profile.to_notebook_iframe()
# profile.to_file(output_file='mimecast_inbound_traffic_process.html')

In [26]:
# df_process.to_csv('mimecast_inbound_traffic_process.csv', index=False)

## Delivery

In [27]:
df_delivery.Dir.value_counts()

df_inbound_del = df_delivery[df_delivery.Dir == 'Inbound'].reset_index(drop=True)
# df_outbound = df_delivery[df_delivery.Dir == 'Outbound'].reset_index(drop=True)
# df_external = df_delivery[df_delivery.Dir == 'External'].reset_index(drop=True)
# df_internal = df_delivery[df_delivery.Dir == 'Internal'].reset_index(drop=True)
# df_inbound_del.columns = df_inbound_del.columns + '_delivery'
df_inbound_del.columns

Inbound     13700
Outbound    10963
Internal    500  
External    392  
Name: Dir, dtype: int64

Index(['Filename', 'datetime', 'aCode', 'acc', 'Delivered', 'IP', 'AttCnt', 'Dir', 'ReceiptAck', 'MsgId', 'Subject', 'Latency', 'Sender', 'Rcpt', 'AttSize', 'Attempt', 'TlsVer', 'Cphr', 'Snt', 'UseTls', 'Route', 'Err', 'RejType', 'RejCode', 'RejInfo'], dtype='object')

In [28]:
# profile = ProfileReport(df_inbound_del, title='Pandas Profiling Report - Mimecast Delivery')
# # # # profile.to_notebook_iframe()
# profile.to_file(output_file='mimecast_inbound_traffic_delivery.html')

In [29]:
# df_inbound_del.to_csv('mimecast_inbound_traffic_delivery.csv', index=False)

In [38]:
print('receipt')
df_inbound_rec.sample(1)
print('process')
df_process.sample(1)
print('delivery')
df_inbound_del.sample(1)

receipt


,Filename,datetime,aCode,acc,SpamLimit,IP,Dir,MsgId,Subject,headerFrom,Sender,Rcpt,SpamInfo,Act,TlsVer,Cphr,SpamScore,RejInfo,RejType,Error,RejCode,Virus
16517,receipt_20200203185428524.log,2020-02-03T18:22:59-0500,Qe6pZuT9MTGVNLs3qjhtOw,CUSA31A369,NaN,40.107.69.120,Inbound,<CY4PR20MB1288C912D6778DD4518F4C65B5000@CY4PR20MB1288.namprd20.prod.outlook.com>,RE: OneBucks Card Status - Julio Debrigard,lshelton@incomm.com,lshelton@incomm.com,pkoehler@usautoforce.com,NaN,Acc,TLSv1.2,TLS_ECDHE_RSA_WITH_AES_256_GCM_SHA384,NaN,NaN,NaN,NaN,NaN,NaN


process


,Filename,datetime,aCode,acc,AttSize,Act,AttCnt,AttNames,MsgSize,MsgId,Hld
17158,process_20200203180615198.log,2020-02-03T17:39:24-0500,R88Zv9ewMi-l9oLb7MKcXg,CUSA31A369,0,Acc,0,"""PMPF_202002031800_3088986.TXT""",12616,<7c4c9ba3fb714c2c95ea1446095be102@COPWS001.usoil.local>,NaN


delivery


,Filename,datetime,aCode,acc,Delivered,IP,AttCnt,Dir,ReceiptAck,MsgId,Subject,Latency,Sender,Rcpt,AttSize,Attempt,TlsVer,Cphr,Snt,UseTls,Route,Err,RejType,RejCode,RejInfo
1054,delivery_20200203135242647.log,2020-02-03T13:25:47-0500,AORD5uqWO5qgivZ4JN6Ylw,CUSA31A369,true,4.28.59.19,0,Inbound,"250 2.6.0 <1340566849.-124488967.1580754038053.JavaMail.mktmail@sjmas03.marketo.org> [InternalId=232112917581828, Hostname=COPWS002.usoil.local] Queued mail for delivery",<1340566849.-124488967.1580754038053.JavaMail.mktmail@sjmas03.marketo.org>,Webinar: Build an enterprise-ready anomaly detection solution,3530,157-gqe-382.0.1878072.0.0.597574.9.52315156-29@bounce.email.microsoft.com,ehyland@usventure.com,0,1,TLSv1.2,TLS_ECDHE_RSA_WITH_AES_256_CBC_SHA384,25596,Yes,Definition for 4.28.59.19,NaN,NaN,NaN,NaN


# Join Data

In [ ]:
# df_joined = df_inbound_del.set_index('aCode_delivery').join(
#     df_inbound_rec.set_index('aCode_receipt'), how='left').join(
#     df_process.set_index('aCode_process'), how='left').reset_index().rename(columns={'index':'aCode'})
# df_joined.shape